# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laguna,38.4210,-121.4238,15.30,72,0,3.09,US,1708123491
1,1,nar'yan-mar,67.6713,53.0870,-21.61,88,13,4.02,RU,1708123491
2,2,adamstown,-25.0660,-130.1015,24.93,79,88,3.95,PN,1708123491
3,3,blackmans bay,-43.0167,147.3167,19.29,72,80,0.45,AU,1708123491
4,4,whitehorse,60.7161,-135.0538,-3.32,63,0,5.66,CA,1708123491


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='City',
    # colorbar=True,
    hover_cols=["City", "Humidity"],
    size='Humidity',
    # title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    geo=True,
    tiles='OSM',
    width=1200,
    height=600
).opts(legend_position='right', fontsize='11px')



# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values (Personal criteria, feel free to change as needed)
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] <= 30) & 
                                (city_data_df['Max Temp'] >= 20) &
                                (city_data_df['Wind Speed'] <= 4.5) &
                                (city_data_df['Humidity'] <= 70) &
                                (city_data_df['Cloudiness'] <= 30)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.sort_values(by='Max Temp', ascending=False).head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
485,485,kone,-21.0595,164.8658,29.71,58,21,3.56,NC,1708123580
106,106,copala,16.6500,-98.9833,28.93,57,4,3.70,MX,1708123510
481,481,villa dolores,-31.9459,-65.1896,28.00,42,0,0.59,AR,1708123579
182,182,phalaborwa,-23.9430,31.1411,27.60,38,14,1.25,ZA,1708123524
222,222,port augusta,-32.5000,137.7667,27.58,21,0,2.55,AU,1708123532


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,kidal,ML,18.4411,1.4078,11,
40,la'ie,US,21.6477,-157.9253,58,
106,copala,MX,16.6500,-98.9833,57,
139,manokwari,ID,-0.8667,134.0833,68,
182,phalaborwa,ZA,-23.9430,31.1411,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000  # Assuming the radius is in meters
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kidal - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
copala - nearest hotel: No hotel found
manokwari - nearest hotel: Hotel Arfak
phalaborwa - nearest hotel: Hans Merensky Hotel & Estate
pakuranga - nearest hotel: Loomb's Hotel Site
malvan - nearest hotel: Hotel Silver Sand
sur - nearest hotel: Sur Hotel
port augusta - nearest hotel: Hotel Commonwealth
port macquarie - nearest hotel: The Observatory
naliya - nearest hotel: No hotel found
wangaratta - nearest hotel: Hotel Pinsent
villa dolores - nearest hotel: Plaza Hotel
kone - nearest hotel: Pacifik Appartels
zouerate - nearest hotel: فندق تيرس زمور


,City,Country,Lat,Lng,Humidity,Hotel Name
14,kidal,ML,18.4411,1.4078,11,No hotel found
40,la'ie,US,21.6477,-157.9253,58,No hotel found
106,copala,MX,16.6500,-98.9833,57,No hotel found
139,manokwari,ID,-0.8667,134.0833,68,Hotel Arfak
182,phalaborwa,ZA,-23.9430,31.1411,38,Hans Merensky Hotel & Estate
200,pakuranga,NZ,-36.8833,174.9167,58,Loomb's Hotel Site
213,malvan,IN,16.0667,73.4667,51,Hotel Silver Sand
215,sur,OM,22.5667,59.5289,56,Sur Hotel
222,port augusta,AU,-32.5000,137.7667,21,Hotel Commonwealth
353,port macquarie,AU,-31.4333,152.9167,70,The Observatory


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    colorbar=True,
    hover_cols=["City", "Humidity", "Hotel Name", "Country"],
    size='Humidity',
    title='Ideal Cities by Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    geo=True,
    tiles='OSM',
    width=800,
    height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)